# Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import least_squares

# Read data

In [ ]:
# Read the CSV file
df = pd.read_csv(
    "/home/m3rc7pi/workspace/arc/log/20240318_calib.csv", 
    header=None, 
    names=["timestamp", "acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"],
    skiprows=0
)
df['timestamp'] = df['timestamp'] / 1e6

# Accelerometer

In [ ]:
# Compute statistics for acceleration data
df['acc_magnitude'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)

print("\nAcceleration Statistics:")
print(df[["acc_x", "acc_y", "acc_z", 'acc_magnitude']].describe())

In [ ]:
# Plot accelerometer data
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['acc_x'], label='acc_x')
axs[0].plot(df['timestamp'], df['acc_y'], label='acc_y')
axs[0].plot(df['timestamp'], df['acc_z'], label='acc_z')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Accelerometer Data')
axs[0].grid(True)
axs[0].legend()

axs[1].plot(df['timestamp'], df['acc_magnitude'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Acceleration [g]')
axs[1].set_title('Magnitude')
axs[1].grid(True)
axs[1].legend()
plt.tight_layout()
plt.show()

## Data

In [ ]:
acc_x = np.concatenate(( df['acc_x'][0:70],df['acc_x'][130:190],df['acc_x'][230:270],df['acc_x'][330:360],df['acc_x'][370:435],df['acc_x'][490:560],df['acc_x'][700:750],df['acc_x'][780:825],df['acc_x'][840:880],df['acc_x'][920:1000],df['acc_x'][1030:1070], df['acc_x'][1130:]))
acc_y = np.concatenate(( df['acc_y'][0:70],df['acc_y'][130:190],df['acc_y'][230:270],df['acc_y'][330:360],df['acc_y'][370:435],df['acc_y'][490:560],df['acc_y'][700:750],df['acc_y'][780:825],df['acc_y'][840:880],df['acc_y'][920:1000],df['acc_y'][1030:1070], df['acc_y'][1130:]))
acc_z = np.concatenate(( df['acc_z'][0:70],df['acc_z'][130:190],df['acc_z'][230:270],df['acc_z'][330:360],df['acc_z'][370:435],df['acc_z'][490:560],df['acc_z'][700:750],df['acc_z'][780:825],df['acc_z'][840:880],df['acc_z'][920:1000],df['acc_z'][1030:1070], df['acc_z'][1130:]))

# Plot accelerometer data
fig, axs = plt.subplots(2, 1, figsize=(15, 6))
axs[0].plot( acc_x,  label='acc_x')
axs[0].plot( acc_y,  label='acc_y')
axs[0].plot( acc_z,  label='acc_z')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Accelerometer Data')
axs[0].grid(True)
axs[0].grid(which='minor')
axs[0].minorticks_on()
axs[0].legend()

mag = acc_x ** 2 + acc_y ** 2 + acc_z ** 2 
axs[1].plot( mag, label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Acceleration [g]')
axs[1].set_title('Magnitude')
axs[1].grid(True)
axs[1].legend()
plt.tight_layout()
plt.show()

## Function to minimize

In [ ]:
def acc_error(theta, x, y, z):
    return (1 - ((x + theta[0])**2 + (y + theta[1])**2 + (z + theta[2])**2))

In [ ]:
# Initial parameter guess
theta_0 = np.array([
    0.0, # x bias
    0.0, # y bias
    0.0, # z bias
])

# Compute solution providing initial guess theta0, x input, and y input
sol = least_squares(
    acc_error, 
    theta_0, 
    args=(acc_x,acc_y,acc_z), 
    method='lm'
)
print(sol.x / (1.0 / (1 << 11)))


In [ ]:
acc_x_corr = acc_x + sol.x[0] 
acc_y_corr = acc_y + sol.x[1] 
acc_z_corr = acc_z + sol.x[2] 

# Plot accelerometer data
fig, axs = plt.subplots(2, 1, figsize=(15, 6))
axs[0].plot( acc_x_corr,  label='acc_x')
axs[0].plot( acc_y_corr,  label='acc_y')
axs[0].plot( acc_z_corr,  label='acc_z')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Accelerometer Data')
axs[0].grid(True)
axs[0].grid(which='minor')
axs[0].minorticks_on()
axs[0].legend()

mag = acc_x_corr ** 2 + acc_y_corr ** 2 + acc_z_corr ** 2 
axs[1].plot( mag, label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Acceleration [g]')
axs[1].set_title('Magnitude')
axs[1].grid(True)
axs[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot accelerometer data
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['acc_x'] + sol.x[0],  label='acc_x')
axs[0].plot(df['timestamp'], df['acc_y'] + sol.x[1],  label='acc_y')
axs[0].plot(df['timestamp'], df['acc_z'] + sol.x[2],  label='acc_z')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Accelerometer Data')
axs[0].grid(True)
axs[0].legend()

mag = (df['acc_x'] + sol.x[0]) ** 2 + (df['acc_y'] + sol.x[1]) ** 2 + (df['acc_z'] + sol.x[2]) ** 2 
axs[1].plot(df['timestamp'], mag, label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Acceleration [g]')
axs[1].set_title('Magnitude')
axs[1].grid(True)
axs[1].legend()
plt.tight_layout()
plt.show()

# Gyroscope

In [ ]:
# Compute statistics for gyro data
gyro_stats = df[["gyro_x", "gyro_y", "gyro_z"]].describe()
print("\nGyro Statistics:")
print(gyro_stats)

In [ ]:
# Plot gyroscope data
plt.figure(figsize=(12, 3))
plt.plot(df['timestamp'], df['gyro_x'], label='gyro_x')
plt.plot(df['timestamp'], df['gyro_y'], label='gyro_y')
plt.plot(df['timestamp'], df['gyro_z'], label='gyro_z')
plt.xlabel('Timestamp')
plt.ylabel('Angular Velocity')
plt.title('Gyroscope Data')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
df['gyro_x_filtered'] = savgol_filter(df['gyro_x'], window_length=51, polyorder=3)

# Compute the difference between gyro_x and gyro_x_filtered
df['gyro_x_noise'] = df['gyro_x'] - df['gyro_x_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['gyro_x'], label='gyro_x')
axs[0].plot(df['timestamp'], df['gyro_x_filtered'], label='gyro_x filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Angular Velocity [rad/s]')
axs[0].set_title('Filtered X Angular Velocity')
axs[0].legend()

axs[1].plot(df['timestamp'], df['gyro_x_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[rad/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.05, 0.05)
axs[1].legend()
plt.tight_layout()
plt.show()

print(df[["gyro_x", "gyro_x_filtered", "gyro_x_noise"]].describe())

In [ ]:
df['gyro_y_filtered'] = savgol_filter(df['gyro_y'], window_length=51, polyorder=3)

# Compute the difference between gyro_y and gyro_y_filtered
df['gyro_y_noise'] = df['gyro_y'] - df['gyro_y_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['gyro_y'], label='gyro_y')
axs[0].plot(df['timestamp'], df['gyro_y_filtered'], label='gyro_y filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Angular Velocity [rad/s]')
axs[0].set_title('Filtered Y Angular Velocity')
axs[0].legend()

axs[1].plot(df['timestamp'], df['gyro_y_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[rad/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.05, 0.05)
axs[1].legend()
plt.tight_layout()
plt.show()

print(df[["gyro_y", "gyro_y_filtered", "gyro_y_noise"]].describe())

In [ ]:
df['gyro_z_filtered'] = savgol_filter(df['gyro_z'], window_length=51, polyorder=3)

# Compute the difference between gyro_z and gyro_z_filtered
df['gyro_z_noise'] = df['gyro_z'] - df['gyro_z_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['gyro_z'], label='gyro_z')
axs[0].plot(df['timestamp'], df['gyro_z_filtered'], label='gyro_z filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Angular Velocity [rad/s]')
axs[0].set_title('Filtered Y Angular Velocity')
axs[0].legend()

axs[1].plot(df['timestamp'], df['gyro_z_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[rad/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.05, 0.05)
axs[1].legend()
plt.tight_layout()
plt.show()

print(df[["gyro_z", "gyro_z_filtered", "gyro_z_noise"]].describe())